## Setup

### Remove notebook Warning

In [3]:
import warnings
warnings.filterwarnings('ignore')

### Auto-reloard and better matplot

In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

### Imports

In [15]:
import cv2
import cvzone
from ultralytics import YOLO
import numpy as np
import time
from IPython.display import Video
from screeninfo import get_monitors
import os
from collections import defaultdict

### Monitor Infos

In [6]:
# Get a list of connected monitors
monitors = get_monitors()

# Get the dimensions of the first monitor
monitor = monitors[0]
monitor_width = monitor.width
monitor_height = monitor.height

## Params

In [19]:
video_name = "P1077418_Balcon_4K25FPS.MP4"

mask_name = "P1077418_Balcon_4K25FPS_MASK.jpg"
apply_mask = False
display_mask = False


## Main

### Model

In [8]:
model = YOLO('Model/yolov8n-face.pt')

### Loading file

In [9]:
#Path video to analyse and check file exist in ./Videos/video_name
file_video_path = os.path.join(os. getcwd(), "Videos", video_name )
if os.path.exists(file_video_path):
    pass
else :
    raise FileExistsError

### Open Video

In [10]:
cap = cv2.VideoCapture(file_video_path)

### Video Infos

In [11]:
fps = cap.get(cv2.CAP_PROP_FPS)
delay = int(1000 / fps)


video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

### Open Mask

In [12]:
if apply_mask :

    mask_path = os.path.join(os. getcwd(), "Videos", mask_name )

    if os.path.exists(mask_path):
        mask = cv2.imread(mask_path)

        mask_gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        _, mask_binary = cv2.threshold(mask_gray, 128, 255, cv2.THRESH_BINARY)

    else:
        raise FileExistsError


### Display Video and detection

In [20]:
# Create a window with the size of the video at the calculated position
cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
cv2.moveWindow("Image", (video_width - monitor_width) // 2, (video_height - monitor_height) // 2)
cv2.resizeWindow("Image", monitor_width, monitor_height)

track_history = defaultdict(lambda: [])


while cap.isOpened():

    success, img = cap.read()
    if success:

        if apply_mask:
            img_ready = cv2.bitwise_and(img, img, mask=mask_binary)

        else :
            img_ready = img


        results = model.track(img_ready, persist=True)

        # Get the boxes and track IDs
        boxes = results[0].boxes.xywh.cpu()

        try :
            track_ids = results[0].boxes.id.int().cpu().tolist()
        except AttributeError:
            track_ids = results[0].boxes.xywh

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Plot the tracks
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))  # x, y center point
            if len(track) > 30:  # retain 90 tracks for 90 frames
                track.pop(0)

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

        # Display the annotated frame
        cv2.imshow("Image", annotated_frame)

        #cv2.imshow("Image", img)

        #End program
        if cv2.waitKey(delay) == ord('q'):
            cv2.destroyAllWindows()
            break


0: 384x640 1 face, 85.7ms
Speed: 4.0ms preprocess, 85.7ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 faces, 29.9ms
Speed: 5.7ms preprocess, 29.9ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 faces, 18.1ms
Speed: 4.2ms preprocess, 18.1ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 faces, 14.5ms
Speed: 3.1ms preprocess, 14.5ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 faces, 20.7ms
Speed: 3.1ms preprocess, 20.7ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 faces, 19.8ms
Speed: 8.4ms preprocess, 19.8ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 faces, 16.4ms
Speed: 6.2ms preprocess, 16.4ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 faces, 17.7ms
Speed: 3.3ms preprocess, 17.7ms inference, 3.6ms postprocess per image at shape (1, 3, 384, 640)



KeyboardInterrupt: 